In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
df= pd.read_csv('MHST_monsties.csv')
df

,No,Monster,HP,Attack,Defence,Speed,Att_Fire,Att_Water,Att_Thunder,Att_Ice,Att_Dragon,Res_Fire,Res_Water,Res_Thunder,Res_Ice,Res_Dragon,Tendency
0,1,Aptonoth,2,1,2,1,3,3,3,3,3,1,1,1,1,5,2
1,2,Velocidrome,1,1,1,4,3,3,3,3,4,3,3,3,1,5,1
2,3,Arzuros,2,1,4,2,3,4,3,3,3,1,5,2,3,3,2
3,4,Yian-Kut-Ku,2,1,3,4,4,3,3,2,3,5,3,3,1,3,3
4,5,Blue Yian-Kut-Ku,2,2,3,4,5,2,3,3,3,5,2,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,60,Silver Rathalos,5,5,4,3,5,1,2,3,5,5,1,2,3,5,2
60,61,White Monoblos,3,5,4,3,3,5,3,1,3,3,5,3,1,3,2
61,62,Molten Tigrex,3,5,4,3,5,2,4,4,4,5,1,3,3,3,2
62,63,Oroshi Kirin,1,4,4,5,2,3,3,5,3,1,1,5,5,5,1


In [3]:
df1 = df.drop(columns=['No'])

In [4]:
# Memisahkan fitur dan target
X = df1.drop(columns=['Monster'])
y = df1['Tendency']

# Preprocessing data (scaling)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Membuat model Decision Tree
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

# Memprediksi dan evaluasi
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00         5
           2       1.00      1.00      1.00         4
           3       1.00      1.00      1.00         4

    accuracy                           1.00        13
   macro avg       1.00      1.00      1.00        13
weighted avg       1.00      1.00      1.00        13



In [5]:
def recommend_monster_v3(opponent_stats, df):
    rankings = []
    
    # Mengidentifikasi elemen resistance terendah dari musuh
    resistance_values = {
        'Fire': opponent_stats['Res_Fire'],
        'Water': opponent_stats['Res_Water'],
        'Thunder': opponent_stats['Res_Thunder'],
        'Ice': opponent_stats['Res_Ice'],
        'Dragon': opponent_stats['Res_Dragon']
    }
    
    # Mendapatkan resistance terendah
    lowest_resistance = min(resistance_values, key=resistance_values.get)
    
    # Iterasi melalui data monster
    for index, row in df.iterrows():
        monster_stats = row.drop('Monster').to_dict()
        score = 0
        alasan = []

        # Prioritas 1: Tendency Counter
        if opponent_stats['Tendency'] == 2 and monster_stats['Tendency'] == 1:  # Power vs Speed
            score += 10  # Speed mengalahkan Power
            alasan.append("Speed counter Power (Tendency)\n")
        elif opponent_stats['Tendency'] == 3 and monster_stats['Tendency'] == 2:  # Technique vs Power
            score += 10  # Power mengalahkan Technique
            alasan.append("Power counter Technique (Tendency)\n")
        elif opponent_stats['Tendency'] == 1 and monster_stats['Tendency'] == 3:  # Speed vs Technique
            score += 10  # Technique mengalahkan Speed
            alasan.append("Technique counter Speed (Tendency)\n")
        else:
            score -= 5
            alasan.append("Tidak meng-counter tendency musuh")

        # Prioritas 2 dan 3: Elemental Counter (Attack vs Resistance)
        # Mengambil attack elemen tertinggi berdasarkan resistance terendah musuh
        attack_values = {
            'Fire': monster_stats['Att_Fire'],
            'Water': monster_stats['Att_Water'],
            'Thunder': monster_stats['Att_Thunder'],
            'Ice': monster_stats['Att_Ice'],
            'Dragon': monster_stats['Att_Dragon']
        }
        
        # Attack tertinggi untuk elemen dengan resistance terendah dari musuh
        highest_attack_value = attack_values[lowest_resistance]
        score += highest_attack_value
        
        alasan.append(f" - Musuh memiliki resistance terendah terhadap {lowest_resistance}\n   - Element monster rekomendasi adalah {lowest_resistance},dengan Attack Element = {highest_attack_value}")

        # Mengumpulkan skor monster dan alasan
        rankings.append((row['Monster'], score, alasan, highest_attack_value))

    # Mengurutkan monster berdasarkan skor tertinggi dan mengambil top 10
    rankings.sort(key=lambda x: x[1], reverse=True)
    top_5_rankings = rankings[:5]
    
    return top_5_rankings


In [6]:
# Statistik monster musuh
opponent_stats = { 
    'HP': 5,
    'Attack': 5,
    'Defence': 4,
    'Speed': 3,
    'Att_Fire': 5,
    'Att_Water': 1,
    'Att_Thunder': 2,
    'Att_Ice': 3,
    'Att_Dragon': 5,
    'Res_Fire': 5,
    'Res_Water': 1,
    'Res_Thunder': 2,
    'Res_Ice': 3,
    'Res_Dragon': 5,
    'Tendency': 2
}

# Mendapatkan ranking rekomendasi monster berdasarkan kriteria yang diinginkan
monster_rankings = recommend_monster_v3(opponent_stats, df)
print("Ranking Rekomendasi Monster (Top 5):")
for rank, (monster, score, alasan, highest_att) in enumerate(monster_rankings, start=1):
    alasan_str = "; ".join(alasan)
    print(f"{rank}. {monster}\n   - Alasan: {alasan_str}\n   - Score: {score}\n")

Ranking Rekomendasi Monster (Top 5):
1. Barroth
   - Alasan: Speed counter Power (Tendency)
;  - Musuh memiliki resistance terendah terhadap Water
   - Element monster rekomendasi adalah Water,dengan Attack Element = 5
   - Score: 15

2. Ivory Lagiacrus
   - Alasan: Speed counter Power (Tendency)
;  - Musuh memiliki resistance terendah terhadap Water
   - Element monster rekomendasi adalah Water,dengan Attack Element = 5
   - Score: 15

3. Lagombi
   - Alasan: Speed counter Power (Tendency)
;  - Musuh memiliki resistance terendah terhadap Water
   - Element monster rekomendasi adalah Water,dengan Attack Element = 4
   - Score: 14

4. Nargacuga
   - Alasan: Speed counter Power (Tendency)
;  - Musuh memiliki resistance terendah terhadap Water
   - Element monster rekomendasi adalah Water,dengan Attack Element = 4
   - Score: 14

5. Green Nargacuga
   - Alasan: Speed counter Power (Tendency)
;  - Musuh memiliki resistance terendah terhadap Water
   - Element monster rekomendasi adalah Wat